In [4]:
import qmcpy as qp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import cProfile
import pstats

In [5]:
def natural_thread(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def natural(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,is_parallel=False)
    points = l.gen_samples(num_ports) 
    return points


In [9]:
n = 10
d_list = [50, 100, 200, 400, 800, 1600]
data = {'index':d_list}
frame = pd.DataFrame(data)
frame = frame.set_index('index')
names = set()
trials = 3

run_list = [(natural, "natural"),(natural_thread,"natural thread")]



for method,method_name in run_list:

    name = method_name.split(" ")

    names.add(name[0])

    times = []
    
    for d in d_list:
        profiler = cProfile.Profile()
        profiler.enable()
        for _ in range(trials):
            method(d, 2 ** n)
        profiler.disable()
        
        stats = pstats.Stats(profiler)
            
        process_time = stats.total_tt

        times.append(process_time)
        
    frame[method_name] = times  
stats.sort_stats('cumulative')

names = list(names)     
 
print(frame) 



        natural  natural thread
index                          
50     0.035683        0.033400
100    0.063313        0.048836
200    0.098791        0.073900
400    0.144441        0.122563
800    0.264514        0.222332
1600   0.499347        0.387467
